This notebook aims to relate COVID cases to pre-existing chronic diseases in the USA by looking at geographical correlations

Currently, just diabetes -- plan to add more

Note: need to control for other factors, like population density, social/economic demographics, etc.

## Datasets Used
Covid county dataset from: https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/

2017 Diabetes county dataset from: https://gis.cdc.gov/grasp/diabetes/DiabetesAtlas.html

In [89]:
import pandas as pd
import matplotlib.pyplot as plt

### Load Datasets

In [90]:
USAFACTS_DIR = 'USAFacts_Covid_by_county/'
COUNTY_CASES_FILENAME = USAFACTS_DIR + 'covid_confirmed_usafacts.csv'
COUNTY_DEATHS_FILENAME = USAFACTS_DIR + 'covid_deaths_usafacts.csv'
COUNTY_POP_FILENAME = USAFACTS_DIR + 'covid_county_population_usafacts.csv'

DIABETES_DIR = 'Diabetes_by_county/'
DIABETES_FILENAME = DIABETES_DIR + 'DiabetesAtlasCountyData.csv'

county_cases = pd.read_csv(COUNTY_CASES_FILENAME) #covid cases by county
county_deaths = pd.read_csv(COUNTY_DEATHS_FILENAME) #covid deaths by county
county_pop = pd.read_csv(COUNTY_POP_FILENAME) #population of counties
county_diabetes = pd.read_csv(DIABETES_FILENAME, skiprows = 2, skipfooter = 1)

<ipython-input-90-8700583f7909>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  county_diabetes = pd.read_csv(DIABETES_FILENAME, skiprows = 2, skipfooter = 1)


In [91]:
#county_cases is a pandas.DataFrame that holds cumulative covid cases per county on daily basis, since 1/22/20

county_cases

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,2242,2267,2283,2304,2328,2351,2385,2417,2435,2456
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,7134,7188,7226,7263,7348,7409,7454,7523,7596,7646
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,1090,1092,1095,1098,1107,1112,1113,1117,1123,1128
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,917,924,926,932,948,961,966,973,978,986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,0,0,0,0,0,0,...,681,692,726,776,869,869,944,991,1075,1075
3191,56039,Teton County,WY,56,0,0,0,0,0,0,...,922,932,977,1007,1025,1025,1066,1108,1178,1178
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,...,542,544,551,568,627,627,664,686,729,729
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,...,185,185,187,190,199,199,203,206,218,218


In [92]:
#county_deaths is pandas.DataFrame that follows same format as county_cases

county_deaths

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,30,30,30,30,30,32,32,36,36,36
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,83,83,83,83,84,84,84,84,84,84
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,9
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,15,16,16,16,17,17,17,17,17,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,4
3191,56039,Teton County,WY,56,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,...,7,7,7,7,7,7,7,7,7,7


In [93]:
county_pop

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394
...,...,...,...,...
3190,56037,Sweetwater County,WY,42343
3191,56039,Teton County,WY,23464
3192,56041,Uinta County,WY,20226
3193,56043,Washakie County,WY,7805


In [94]:
#the 2017 diabetes rate by county -- percentage is between 0 and 100

county_diabetes

,County,State,CountyFIPS,Percentage,Lower Limit,Upper Limit
0,Bedford City,Virginia,51515,No Data,No Data,No Data
1,Shannon County,South Dakota,46113,No Data,No Data,No Data
2,Wrangell-Petersburg Census Area,Alaska,2280,No Data,No Data,No Data
3,Wade Hampton Census Area,Alaska,2270,No Data,No Data,No Data
4,Skagway-Hoonah-Angoon,Alaska,2232,No Data,No Data,No Data
...,...,...,...,...,...,...
3221,Jones County,North Carolina,37103,24.7,12.8,41.6
3222,Woodruff County,Arkansas,5147,25.3,11.4,46.8
3223,Bullock County,Alabama,1011,27.9,14.9,43.7
3224,Nowata County,Oklahoma,40105,27.9,14,44.3


### Quick EDA on COVID county dataset

In [95]:
#check datatypes of covid_cases
#note: may want to downsize these dtypes to be smaller, use less bits (int32?)

county_cases.dtypes

countyFIPS      int64
County Name    object
State          object
stateFIPS       int64
1/22/20         int64
                ...  
11/11/20        int64
11/12/20        int64
11/13/20        int64
11/14/20        int64
11/15/20        int64
Length: 303, dtype: object

In [96]:
#countyFIPS is the Federal Information Processing Standard code -- unique for each county
#stateFIPS is similar
#State is two-letter state code (ie CA, WY)
#County Name is the name of the county, obviously
#every other column has number of cumulative cases in that county on that date

county_cases.columns

Index(['countyFIPS', 'County Name', 'State', 'stateFIPS', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '11/6/20', '11/7/20', '11/8/20', '11/9/20', '11/10/20', '11/11/20',
       '11/12/20', '11/13/20', '11/14/20', '11/15/20'],
      dtype='object', length=303)

In [97]:
#total number of counties represented in this dataset
county_cases['countyFIPS'].nunique()

3146

In [98]:
#get number of counties per state
county_cases.groupby('State')['countyFIPS'].count()

State
AK     31
AL     68
AR     76
AZ     16
CA     60
CO     65
CT      9
DC      1
DE      4
FL     68
GA    160
HI      6
IA    100
ID     45
IL    103
IN     93
KS    106
KY    121
LA     65
MA     15
MD     25
ME     17
MI     84
MN     88
MO    116
MS     83
MT     57
NC    101
ND     54
NE     94
NH     11
NJ     22
NM     34
NV     18
NY     64
OH     89
OK     78
OR     37
PA     68
RI      6
SC     47
SD     67
TN     96
TX    255
UT     30
VA    134
VT     15
WA     40
WI     73
WV     56
WY     24
Name: countyFIPS, dtype: int64

In [99]:
#get number of cases per state on each date
county_cases.drop(['countyFIPS','County Name','stateFIPS'], axis = 1).groupby('State').sum()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20
State,,,,,,,,,,,,,,,,,,,,,
AK,0,0,0,0,0,0,0,0,0,0,...,17597,18198,18716,19187,19712,20207,20688,21275,22014,22662
AL,0,0,0,0,0,0,0,0,0,0,...,200713,202482,203687,204856,206567,208636,210637,213617,215843,217822
AR,0,0,0,0,0,0,0,0,0,0,...,119230,120828,121866,122811,124235,126197,128006,130317,132163,133040
AZ,0,0,0,0,1,1,1,1,1,1,...,254764,257384,259264,259699,263133,265163,266562,269577,273053,275436
CA,0,0,0,0,2,3,3,4,4,4,...,963768,969090,973743,981493,990096,995487,1005994,1015809,1023575,1030112
CO,0,0,0,0,0,0,0,0,0,0,...,124466,127967,130984,134536,138426,142399,147597,154037,159233,163417
CT,0,0,0,0,0,0,0,0,0,0,...,78125,78125,78125,81463,82987,84741,85899,88645,88645,88645
DC,0,0,0,0,0,0,0,0,0,0,...,17792,17891,18001,18087,18173,18379,18507,18666,18814,18977
DE,0,0,0,0,0,0,0,0,0,0,...,26035,26283,26596,26908,27112,27353,27546,28016,28396,28803


### Join covid cases dataset with chronic disease datasets

Note: work on reducing memory size of dataframes by downsizing big dtypes into smaller dtypes -- may be issue in future

In [100]:
#preprocess chronic disease datasets

#diabetes dataset
temp = county_diabetes.drop(['County','State'], axis = 1) #redundant info, only need CountyFIPS for identification
cols = list(temp.columns)
cols[0] = 'countyFIPS' #rename to match column name in covid dataset
cols[1:] = ['Diabetes_' + c for c in cols[1:]]
temp.columns = cols #rename columns so that it's clear that they came from diabetes dataset
temp.iloc[:,1:] = temp.iloc[:,1:].apply(lambda col: pd.to_numeric(col, errors='coerce', downcast = 'float'), axis = 1) #convert to numeric datatype
county_diabetes = temp

In [104]:
main_df = county_cases.merge(county_diabetes,how='outer', on = 'countyFIPS').dropna()
main_df

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,Diabetes_Percentage,Diabetes_Lower Limit,Diabetes_ Upper Limit
50,1001,Autauga County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2304.0,2328.0,2351.0,2385.0,2417.0,2435.0,2456.0,11.4,8.3,15.200000
51,1003,Baldwin County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7263.0,7348.0,7409.0,7454.0,7523.0,7596.0,7646.0,8.2,6.9,9.700000
52,1005,Barbour County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1098.0,1107.0,1112.0,1113.0,1117.0,1123.0,1128.0,15.1,9.9,22.700001
53,1007,Bibb County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,932.0,948.0,961.0,966.0,973.0,978.0,986.0,12.2,7.8,18.500000
54,1009,Blount County,AL,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2335.0,2378.0,2400.0,2429.0,2488.0,2518.0,2549.0,12.2,8.9,16.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56.0,0.0,0.0,0.0,0.0,0.0,0.0,...,776.0,869.0,869.0,944.0,991.0,1075.0,1075.0,8.0,6.6,9.900000
3191,56039,Teton County,WY,56.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1007.0,1025.0,1025.0,1066.0,1108.0,1178.0,1178.0,2.2,1.3,3.200000
3192,56041,Uinta County,WY,56.0,0.0,0.0,0.0,0.0,0.0,0.0,...,568.0,627.0,627.0,664.0,686.0,729.0,729.0,9.5,7.1,12.300000
3193,56043,Washakie County,WY,56.0,0.0,0.0,0.0,0.0,0.0,0.0,...,190.0,199.0,199.0,203.0,206.0,218.0,218.0,8.2,5.5,12.200000


In [108]:
#check for correlation between diabetes percentage and covid cases

main_df.drop(['countyFIPS', 'stateFIPS'], axis = 1).corr().iloc[:, -3:]

,Diabetes_Percentage,Diabetes_Lower Limit,Diabetes_ Upper Limit
1/22/20,-0.014613,0.001933,-0.022569
1/23/20,-0.014613,0.001933,-0.022569
1/24/20,-0.016015,0.008004,-0.028873
1/25/20,-0.016015,0.008004,-0.028873
1/26/20,-0.026580,0.010690,-0.045738
...,...,...,...
11/14/20,-0.070520,0.096053,-0.180603
11/15/20,-0.070934,0.095867,-0.181096
Diabetes_Percentage,1.000000,0.826828,0.857917
Diabetes_Lower Limit,0.826828,1.000000,0.436664


hmm -- based on correlation table, it seems that there's either no correlation or very weak between COVID cases and diabetes; but what about COVID deaths and diabetes?

Also, check out urban/rural disparity -- may shed some light on if there's interaction effects with diabetes